In [8]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

import pandas as pd
import numpy as np
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.set_random_seed(seed)


features =20
filename="./dataset/20200605_v1.csv"

# 데이터 읽어 들이기--- (※1)
#ci_year	ci_term	si_grad	si_ent_gubun	agescale	si_sex	si_last_hak	schoarYn	
#accept_point	avg_logincnt	avg_att_time	avg_att_rate	avg_bbscnt	avg_startlogin	entYn	classcnt	
#avg_te_avg	avg_te_passrate	gradcnt	counsel	stdResult

df_pre=pd.read_csv(filename,encoding="cp949", header=0) # CSV파일을 불러오는 함수를 이용
#df_pre=pd.read_csv('./dataset/20200612_app_point.csv',encoding="cp949", header=0) # CSV파일을 불러오는 함수를 이용

#df_pre_new=df_pre[df_pre.stdResult!=3] #졸업생은 제외



In [9]:

df_pre


,ci_year,ci_term,si_grad,si_ent_gubun,agescale,si_sex,si_last_hak,schoarYn,accept_point,avg_logincnt,...,avg_att_rate,avg_bbscnt,avg_startlogin,entYn,classcnt,pre_avg_te_avg,pre_avg_te_passrate,pre_gradcnt,counsel,stdResult
0,2015,1,4,1,60,1,1,1,0,87,...,99.56,0,38,0,5,3.50,100,7,0,1
1,2015,1,4,1,30,2,1,1,0,26,...,85.52,0,18,0,4,2.00,100,10,0,1
2,2015,1,4,1,40,2,2,0,0,26,...,100.00,0,13,0,3,2.50,100,10,0,1
3,2015,1,4,1,20,1,2,0,0,8,...,84.21,0,5,0,6,2.00,100,8,0,1
4,2015,1,4,1,30,1,2,0,0,5,...,81.58,0,4,0,1,0.00,0,12,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58083,2019,2,4,2,50,1,3,1,70,74,...,100.00,0,30,0,6,3.42,100,1,0,0
58084,2019,2,4,2,50,2,3,1,70,50,...,99.72,0,20,0,8,3.67,100,1,0,0
58085,2019,2,4,2,40,2,3,1,70,63,...,100.00,6,22,0,6,3.33,100,1,0,0
58086,2019,2,4,2,40,2,2,1,70,98,...,100.00,12,47,0,8,3.67,100,1,0,0


In [ ]:
# 데이터 내부의 기호를 숫자로 변환하기--- (※2)
df = df_pre.sample(frac=1)

dataset = df.values
X = dataset[:,0:features]
Y = dataset[:,features]

# 학습 전용과 테스트 전용 데이터로 나누기 --- (※3)
#테스트, 검증데이터 분할 7:3 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

#데이터 정규화
X_train, X_test = X_train / 255, X_test / 255

from keras.layers import Dense, Dropout, Activation
# 모델 설정
model = Sequential()
model.add(Dense(28, input_dim=features, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#모델 컴파일
model.compile(loss='mean_squared_error',
           optimizer='adam',
           # metrics=['accuracy'])
           metrics=['accuracy',recall_m,precision_m,f1_m])

# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

# 모델 실행 및 저장
history = model.fit(X_train, Y_train, validation_split=0.20, epochs=500, batch_size=100, 
                    callbacks=[early_stopping_callback])


# 결과 출력
print("\n epoches-500, bat_size=100")
print("\n 학습중단 + 모델 성능개선 : arly_stopping_callback:")
print("\n 예측정확도: %.4f" % (model.evaluate(X_test, Y_test)[1]))


#테스트 데이터 검증
loss, accuracy, recall,precision,f1_socre  = model.evaluate(X_test, Y_test)
#accuracy  = model.evaluate(X_test, Y_test)

print("\n accuracy, precision, recall, f1_score")
print("\n  %.4f, %.4f, %4f, %.4f" % (accuracy, precision ,recall, f1_socre))

#print("\n accuracy")
#print("\n  %.4f % (accuracy)")

Train on 32528 samples, validate on 8133 samples
Epoch 1/500
32528/32528 [==============================] - 1s 16us/step - loss: 0.2223 - accuracy: 0.6671 - recall_m: 0.9107 - precision_m: 0.6434 - f1_m: 0.7524 - val_loss: 0.2053 - val_accuracy: 0.7001 - val_recall_m: 1.0000 - val_precision_m: 0.7003 - val_f1_m: 0.8230
Epoch 2/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.2019 - accuracy: 0.7035 - recall_m: 0.9997 - precision_m: 0.7038 - f1_m: 0.8252 - val_loss: 0.2034 - val_accuracy: 0.7052 - val_recall_m: 0.9924 - val_precision_m: 0.7063 - val_f1_m: 0.8244
Epoch 3/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.2001 - accuracy: 0.7068 - recall_m: 0.9937 - precision_m: 0.7078 - f1_m: 0.8258 - val_loss: 0.2007 - val_accuracy: 0.7033 - val_recall_m: 0.9862 - val_precision_m: 0.7069 - val_f1_m: 0.8226
Epoch 4/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.1978 - accuracy: 0.7104 - recall_m: 0.9862 - precisio

Epoch 31/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.1350 - accuracy: 0.8132 - recall_m: 0.8922 - precision_m: 0.8507 - f1_m: 0.8698 - val_loss: 0.1341 - val_accuracy: 0.8140 - val_recall_m: 0.8707 - val_precision_m: 0.8647 - val_f1_m: 0.8668
Epoch 32/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.1358 - accuracy: 0.8105 - recall_m: 0.8896 - precision_m: 0.8502 - f1_m: 0.8678 - val_loss: 0.1328 - val_accuracy: 0.8168 - val_recall_m: 0.8806 - val_precision_m: 0.8609 - val_f1_m: 0.8699
Epoch 33/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1347 - accuracy: 0.8117 - recall_m: 0.8897 - precision_m: 0.8512 - f1_m: 0.8684 - val_loss: 0.1373 - val_accuracy: 0.8106 - val_recall_m: 0.9273 - val_precision_m: 0.8246 - val_f1_m: 0.8722
Epoch 34/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1348 - accuracy: 0.8126 - recall_m: 0.8911 - precision_m: 0.8517 - f1_m: 0.8692 - val_loss: 0.1338

Epoch 61/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.1271 - accuracy: 0.8230 - recall_m: 0.8981 - precision_m: 0.8579 - f1_m: 0.8762 - val_loss: 0.1294 - val_accuracy: 0.8204 - val_recall_m: 0.8566 - val_precision_m: 0.8831 - val_f1_m: 0.8688
Epoch 62/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.1276 - accuracy: 0.8216 - recall_m: 0.8978 - precision_m: 0.8566 - f1_m: 0.8753 - val_loss: 0.1252 - val_accuracy: 0.8260 - val_recall_m: 0.9094 - val_precision_m: 0.8519 - val_f1_m: 0.8789
Epoch 63/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.1267 - accuracy: 0.8221 - recall_m: 0.8966 - precision_m: 0.8577 - f1_m: 0.8755 - val_loss: 0.1489 - val_accuracy: 0.7671 - val_recall_m: 0.9673 - val_precision_m: 0.7637 - val_f1_m: 0.8529
Epoch 64/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.1268 - accuracy: 0.8226 - recall_m: 0.8986 - precision_m: 0.8572 - f1_m: 0.8760 - val_loss: 0.1248

32528/32528 [==============================] - 0s 11us/step - loss: 0.1237 - accuracy: 0.8285 - recall_m: 0.9042 - precision_m: 0.8604 - f1_m: 0.8805 - val_loss: 0.1228 - val_accuracy: 0.8351 - val_recall_m: 0.9105 - val_precision_m: 0.8612 - val_f1_m: 0.8846
Epoch 92/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1250 - accuracy: 0.8273 - recall_m: 0.9054 - precision_m: 0.8594 - f1_m: 0.8800 - val_loss: 0.1217 - val_accuracy: 0.8355 - val_recall_m: 0.9261 - val_precision_m: 0.8514 - val_f1_m: 0.8865
Epoch 93/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1241 - accuracy: 0.8277 - recall_m: 0.9056 - precision_m: 0.8589 - f1_m: 0.8802 - val_loss: 0.1399 - val_accuracy: 0.7917 - val_recall_m: 0.9536 - val_precision_m: 0.7919 - val_f1_m: 0.8645
Epoch 94/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1239 - accuracy: 0.8305 - recall_m: 0.9076 - precision_m: 0.8596 - f1_m: 0.8816 - val_loss: 0.1244 - val_accura

Epoch 121/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1184 - accuracy: 0.8412 - recall_m: 0.9127 - precision_m: 0.8687 - f1_m: 0.8891 - val_loss: 0.1183 - val_accuracy: 0.8438 - val_recall_m: 0.9343 - val_precision_m: 0.8554 - val_f1_m: 0.8925
Epoch 122/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1187 - accuracy: 0.8408 - recall_m: 0.9127 - precision_m: 0.8687 - f1_m: 0.8890 - val_loss: 0.1163 - val_accuracy: 0.8458 - val_recall_m: 0.9165 - val_precision_m: 0.8699 - val_f1_m: 0.8919
Epoch 123/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1184 - accuracy: 0.8415 - recall_m: 0.9115 - precision_m: 0.8704 - f1_m: 0.8894 - val_loss: 0.1219 - val_accuracy: 0.8346 - val_recall_m: 0.9420 - val_precision_m: 0.8410 - val_f1_m: 0.8880
Epoch 124/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1192 - accuracy: 0.8402 - recall_m: 0.9113 - precision_m: 0.8689 - f1_m: 0.8882 - val_loss: 0.

...

Epoch 181/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1124 - accuracy: 0.8526 - recall_m: 0.9193 - precision_m: 0.8773 - f1_m: 0.8970 - val_loss: 0.1147 - val_accuracy: 0.8517 - val_recall_m: 0.9382 - val_precision_m: 0.8619 - val_f1_m: 0.8979
Epoch 182/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1146 - accuracy: 0.8487 - recall_m: 0.9180 - precision_m: 0.8743 - f1_m: 0.8944 - val_loss: 0.1129 - val_accuracy: 0.8480 - val_recall_m: 0.8994 - val_precision_m: 0.8852 - val_f1_m: 0.8915
Epoch 183/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1121 - accuracy: 0.8530 - recall_m: 0.9205 - precision_m: 0.8774 - f1_m: 0.8976 - val_loss: 0.1116 - val_accuracy: 0.8533 - val_recall_m: 0.9242 - val_precision_m: 0.8733 - val_f1_m: 0.8975
Epoch 184/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1119 - accuracy: 0.8525 - recall_m: 0.9194 - precision_m: 0.8771 - f1_m: 0.8969 - val_loss: 0.

Epoch 211/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1114 - accuracy: 0.8530 - recall_m: 0.9192 - precision_m: 0.8785 - f1_m: 0.8974 - val_loss: 0.1137 - val_accuracy: 0.8537 - val_recall_m: 0.9377 - val_precision_m: 0.8645 - val_f1_m: 0.8991
Epoch 212/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1119 - accuracy: 0.8530 - recall_m: 0.9192 - precision_m: 0.8779 - f1_m: 0.8972 - val_loss: 0.1117 - val_accuracy: 0.8555 - val_recall_m: 0.9160 - val_precision_m: 0.8819 - val_f1_m: 0.8980
Epoch 213/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1107 - accuracy: 0.8533 - recall_m: 0.9199 - precision_m: 0.8778 - f1_m: 0.8976 - val_loss: 0.1111 - val_accuracy: 0.8531 - val_recall_m: 0.9067 - val_precision_m: 0.8862 - val_f1_m: 0.8957
Epoch 214/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1113 - accuracy: 0.8527 - recall_m: 0.9190 - precision_m: 0.8777 - f1_m: 0.8970 - val_loss: 0.

Epoch 241/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1098 - accuracy: 0.8541 - recall_m: 0.9198 - precision_m: 0.8788 - f1_m: 0.8981 - val_loss: 0.1200 - val_accuracy: 0.8447 - val_recall_m: 0.9338 - val_precision_m: 0.8571 - val_f1_m: 0.8931
Epoch 242/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1117 - accuracy: 0.8519 - recall_m: 0.9184 - precision_m: 0.8777 - f1_m: 0.8966 - val_loss: 0.1157 - val_accuracy: 0.8458 - val_recall_m: 0.9389 - val_precision_m: 0.8549 - val_f1_m: 0.8944
Epoch 243/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1116 - accuracy: 0.8527 - recall_m: 0.9196 - precision_m: 0.8779 - f1_m: 0.8972 - val_loss: 0.1406 - val_accuracy: 0.7986 - val_recall_m: 0.9628 - val_precision_m: 0.7934 - val_f1_m: 0.8692
Epoch 244/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1111 - accuracy: 0.8523 - recall_m: 0.9194 - precision_m: 0.8774 - f1_m: 0.8969 - val_loss: 0.

Epoch 271/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.1089 - accuracy: 0.8551 - recall_m: 0.9200 - precision_m: 0.8799 - f1_m: 0.8987 - val_loss: 0.1082 - val_accuracy: 0.8554 - val_recall_m: 0.9287 - val_precision_m: 0.8726 - val_f1_m: 0.8992
Epoch 272/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1094 - accuracy: 0.8551 - recall_m: 0.9202 - precision_m: 0.8800 - f1_m: 0.8988 - val_loss: 0.1084 - val_accuracy: 0.8552 - val_recall_m: 0.9201 - val_precision_m: 0.8784 - val_f1_m: 0.8982
Epoch 273/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1099 - accuracy: 0.8546 - recall_m: 0.9203 - precision_m: 0.8793 - f1_m: 0.8984 - val_loss: 0.1088 - val_accuracy: 0.8554 - val_recall_m: 0.9285 - val_precision_m: 0.8727 - val_f1_m: 0.8991
Epoch 274/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1089 - accuracy: 0.8555 - recall_m: 0.9197 - precision_m: 0.8804 - f1_m: 0.8987 - val_loss: 0.

Epoch 301/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1089 - accuracy: 0.8555 - recall_m: 0.9205 - precision_m: 0.8795 - f1_m: 0.8987 - val_loss: 0.1120 - val_accuracy: 0.8569 - val_recall_m: 0.9381 - val_precision_m: 0.8678 - val_f1_m: 0.9009
Epoch 302/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1085 - accuracy: 0.8553 - recall_m: 0.9196 - precision_m: 0.8801 - f1_m: 0.8986 - val_loss: 0.1152 - val_accuracy: 0.8504 - val_recall_m: 0.8886 - val_precision_m: 0.8967 - val_f1_m: 0.8920
Epoch 303/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1089 - accuracy: 0.8547 - recall_m: 0.9197 - precision_m: 0.8791 - f1_m: 0.8980 - val_loss: 0.1141 - val_accuracy: 0.8554 - val_recall_m: 0.9503 - val_precision_m: 0.8582 - val_f1_m: 0.9014
Epoch 304/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1081 - accuracy: 0.8561 - recall_m: 0.9213 - precision_m: 0.8801 - f1_m: 0.8993 - val_loss: 0.